In [1]:
#Import the NFL play by play data
import pandas as pd 

#Enter desired years of data
YEARS = [2019,2018,2017]

data = pd.DataFrame()

for i in YEARS:  
    #low_memory=False eliminates a warning
    i_data = pd.read_csv('https://github.com/nflverse/nflfastR-data/blob/master/data/' \
                         'play_by_play_' + str(i) + '.csv.gz?raw=True',
                         compression='gzip', low_memory=False)

    #sort=True eliminates a warning and alphabetically sorts columns
    data = data.append(i_data, sort=True)

#Give each row a unique index
data.reset_index(drop=True, inplace=True)

In [2]:
#pd.options.display.max_columns = None
#pd.options.display.max_rows = None

In [3]:
#list(data.columns)

In [4]:
data.info()
#data['down'].head(20)
#new_data.isna().head().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143905 entries, 0 to 143904
Columns: 370 entries, aborted_play to yrdln
dtypes: float64(177), int64(38), object(155)
memory usage: 406.2+ MB


In [ ]:
only_third_downs = data['down' == 3]

In [ ]:
for x in new_data:
    missing = x.isna().sum()
    if missing > len(data.index)/2:
        new_data.drop(x, inplace=True)        
        

- keep only rows that are 3rd down plays
- need results from first and second downs into a column
- every third down has it own play ID
- seperate data frame for 3rd downs
- keep only column that i see as valuable in current state or any columns that may be usful if i change them 
- a way to reduce the number of columns is look at the percentage of missing values
    - if column is missing more than 50% of values then it is hard to do anything with column
    